<center>
    <h1> Assignment 4a </h1>
    <h3> Team: RAIR Data </h3>
</center>

Using the same database from assignment 3b, tables were already normalized to 3NF

In [1]:
from DATA225utils import make_connection, dataframe_query

In [2]:
# try to connect to the database, other wise create the database and connect
try:
    conn = make_connection(config_file = './hospital.ini')
except:
    from create_schema import makedb
    makedb()
    conn = make_connection(config_file = './hospital.ini')

cursor = conn.cursor()

Resetting database

In [3]:
cursor.execute("DROP TABLE IF EXISTS Doctor_Contact")
cursor.execute("DROP TABLE IF EXISTS Stays")
cursor.execute("DROP TABLE IF EXISTS Cared_by")
cursor.execute("DROP TABLE IF EXISTS Hospital_stay")
cursor.execute("DROP TABLE IF EXISTS Ward")
cursor.execute("DROP TABLE IF EXISTS Doctor")
cursor.execute("DROP TABLE IF EXISTS Patient_Contact")
cursor.execute("DROP TABLE IF EXISTS Department")
cursor.execute("DROP TABLE IF EXISTS Building")
cursor.execute("DROP TABLE IF EXISTS Patient")

Creating all tables, including named foreign key constraints

In [4]:
cursor.execute("""
  CREATE TABLE Patient
  (
    Name VARCHAR(50) NOT NULL,
    Sex VARCHAR(1) NOT NULL,
    Date_of_birth DATE NOT NULL,
    Address VARCHAR(50),
    ID INT NOT NULL,
    PRIMARY KEY (ID)
  );
""")

cursor.execute("""
  CREATE TABLE Building
  (
    ID INT NOT NULL,
    street VARCHAR(50) NOT NULL,
    building_number INT NOT NULL,
    PRIMARY KEY (ID)
  );
""")

cursor.execute("""
  CREATE TABLE Department
  (
    ID INT NOT NULL,
    Type VARCHAR(50) NOT NULL,
    Building_ID INT NOT NULL,
    PRIMARY KEY (ID),
    CONSTRAINT dept_in_building FOREIGN KEY (Building_ID) REFERENCES Building(ID)
  );
""")

cursor.execute("""
  CREATE TABLE Patient_Contact
  (
    Contact VARCHAR(10) NOT NULL,
    Patient_ID INT NOT NULL,
    PRIMARY KEY (Contact, Patient_ID),
    CONSTRAINT patient_contact_info FOREIGN KEY (Patient_ID) REFERENCES Patient(ID)
  );
""")

cursor.execute("""
  CREATE TABLE Doctor
  (
    Name VARCHAR(50) NOT NULL,
    Sex VARCHAR(1) NOT NULL,
    Date_of_birth DATE NOT NULL,
    Specialization VARCHAR(50) NOT NULL,
    ID INT NOT NULL,
    Department_ID INT NOT NULL,
    PRIMARY KEY (ID),
    CONSTRAINT doctor_belongs_department FOREIGN KEY (Department_ID) REFERENCES Department(ID)
  );
""")

cursor.execute("""
  CREATE TABLE Ward
  (
    ID INT NOT NULL,
    number_of_beds INT NOT NULL,
    Building_ID INT NOT NULL,
    PRIMARY KEY (ID),
    CONSTRAINT ward_in_building FOREIGN KEY (Building_ID) REFERENCES Building(ID)
  );
""")

cursor.execute("""
  CREATE TABLE Hospital_stay
  (
    ID INT NOT NULL,
    Admit_date DATE NOT NULL,
    Discharge_date DATE NOT NULL,
    Ward_ID INT NOT NULL,
    PRIMARY KEY (ID),
    CONSTRAINT stay_in_ward FOREIGN KEY (Ward_ID) REFERENCES Ward(ID)
  );
""")

cursor.execute("""
  CREATE TABLE Cared_by
  (
    Patient_ID INT NOT NULL,
    Doctor_ID INT NOT NULL,
    PRIMARY KEY (Patient_ID, Doctor_ID),
    CONSTRAINT patient_cared FOREIGN KEY (Patient_ID) REFERENCES Patient(ID),
    CONSTRAINT doctor_cares FOREIGN KEY (Doctor_ID) REFERENCES Doctor(ID)
  );
""")

cursor.execute("""
  CREATE TABLE Stays
  (
    Patient_ID INT NOT NULL,
    Ward_ID INT NOT NULL,
    PRIMARY KEY (Patient_ID, Ward_ID),
    CONSTRAINT patient_stays FOREIGN KEY (Patient_ID) REFERENCES Patient(ID),
    CONSTRAINT ward_used FOREIGN KEY (Ward_ID) REFERENCES Ward(ID)
  );
""")

cursor.execute("""
  CREATE TABLE Doctor_Contact
  (
    Contact VARCHAR(10) NOT NULL,
    Doctor_ID INT NOT NULL,
    PRIMARY KEY (Contact, Doctor_ID)
  );
""")

Using ALTER TABLE command to create a named constraint on one table

In [5]:
cursor.execute("""
  ALTER TABLE Doctor_Contact
  ADD CONSTRAINT doctor_contact_info FOREIGN KEY (Doctor_ID) REFERENCES Doctor(ID)
""")

Inserting data

In [6]:
cursor.execute(""" 
  INSERT INTO `building` VALUES 
  (1,'main street',10),
  (2,'second street',12),
  (3,'Down Street',47),
  (7,'Banana Street',77),
  (15,'Apple Street',22);
""")

cursor.execute(""" 
  INSERT INTO `department` VALUES 
  (1,'Pharmacy',1),
  (2,'Radiology',3),
  (3,'Neurology',7),
  (4,'Emergency',2),
  (5,'Cancer Research',15);
""")

cursor.execute(""" 
  INSERT INTO `doctor` VALUES 
  ('John Watson','M','1990-08-07','Field Doctor',1,1),
  ('Meredith Grey','F','1987-03-22','Surgeon',2,2),
  ('Heshian Uta','M','1980-01-01','Family Doctor',17,3),
  ('Thomas','M','1977-02-02','Pediatrician',29,2),
  ('Susan','F','1965-09-09','Surgeon',33,4);
""")

cursor.execute(""" 
  INSERT INTO `patient` VALUES 
  ('Sherlock Holmes','M','1991-01-06','221-B Baker Street',1),
  ('John Apple','M','1983-04-22','4353 Apple Street',2),
  ('Harry','M','1977-01-05',NULL,3),
  ('John B','M','1991-03-05','336B',19),
  ('Alex C','M','1996-09-08','3311A',20);
""")

cursor.execute(""" 
  INSERT INTO `cared_by` VALUES 
  (1,1),
  (3,2),
  (19,17),
  (20,29),
  (2,33);
""")

cursor.execute(""" 
  INSERT INTO `ward` VALUES 
  (1,20,2),
  (2,44,2),
  (3,50,3),
  (5,10,3),
  (20,35,2);
""")

cursor.execute(""" 
  INSERT INTO `doctor_contact` VALUES 
  ('0101010101',1),
  ('4441114433',1),
  ('4710239118',2),
  ('4109283714',17),
  ('1423987912',33);
""")

cursor.execute(""" 
  INSERT INTO `hospital_stay` VALUES 
  (1,'2023-09-11','2023-09-22',1),
  (2,'2022-11-11','2023-01-01',20),
  (3,'2023-01-01','2023-01-02',2),
  (4,'2023-02-02','2023-03-03',3),
  (5,'2022-01-22','2022-01-24',5);
""")

cursor.execute(""" 
  INSERT INTO `patient_contact` VALUES 
  ('5105105100',1),
  ('0300881429',2),
  ('1231231234',2),
  ('3011111111',19),
  ('4001122221',20);
""")

cursor.execute(""" 
  INSERT INTO `stays` VALUES 
  (2,1),
  (3,1),
  (1,2),
  (19,2),
  (20,3);
""")

conn.commit()

Performing select queries to show data from all the tables

In [7]:
def display_df(conn, sql):
    _, df = dataframe_query(conn, sql)
    display(df)

In [8]:
display_df(conn, "select * from building")

,ID,street,building_number
0,1,main street,10
1,2,second street,12
2,3,Down Street,47
3,7,Banana Street,77
4,15,Apple Street,22


In [9]:
display_df(conn, "select * from cared_by")

,Patient_ID,Doctor_ID
0,1,1
1,3,2
2,19,17
3,20,29
4,2,33


In [10]:
display_df(conn, "select * from department")

,ID,Type,Building_ID
0,1,Pharmacy,1
1,2,Radiology,3
2,3,Neurology,7
3,4,Emergency,2
4,5,Cancer Research,15


In [11]:
display_df(conn, "select * from doctor")

,Name,Sex,Date_of_birth,Specialization,ID,Department_ID
0,John Watson,M,1990-08-07,Field Doctor,1,1
1,Meredith Grey,F,1987-03-22,Surgeon,2,2
2,Heshian Uta,M,1980-01-01,Family Doctor,17,3
3,Thomas,M,1977-02-02,Pediatrician,29,2
4,Susan,F,1965-09-09,Surgeon,33,4


In [12]:
display_df(conn, "select * from doctor_contact")

,Contact,Doctor_ID
0,0101010101,1
1,4441114433,1
2,4710239118,2
3,4109283714,17
4,1423987912,33


In [13]:
display_df(conn, "select * from hospital_stay")

,ID,Admit_date,Discharge_date,Ward_ID
0,1,2023-09-11,2023-09-22,1
1,2,2022-11-11,2023-01-01,20
2,3,2023-01-01,2023-01-02,2
3,4,2023-02-02,2023-03-03,3
4,5,2022-01-22,2022-01-24,5


In [14]:
display_df(conn, "select * from patient")

,Name,Sex,Date_of_birth,Address,ID
0,Sherlock Holmes,M,1991-01-06,221-B Baker Street,1
1,John Apple,M,1983-04-22,4353 Apple Street,2
2,Harry,M,1977-01-05,None,3
3,John B,M,1991-03-05,336B,19
4,Alex C,M,1996-09-08,3311A,20


In [15]:
display_df(conn, "select * from patient_contact")

,Contact,Patient_ID
0,5105105100,1
1,0300881429,2
2,1231231234,2
3,3011111111,19
4,4001122221,20


In [16]:
display_df(conn, "select * from stays")

,Patient_ID,Ward_ID
0,2,1
1,3,1
2,1,2
3,19,2
4,20,3


In [17]:
display_df(conn, "select * from ward")

,ID,number_of_beds,Building_ID
0,1,20,2
1,2,44,2
2,3,50,3
3,5,10,3
4,20,35,2


In [18]:
cursor.close()
conn.close()